In [5]:
import cv2
import numpy as np
import time
import os


class MonoVO():

    def __init__(self,
                 image_path,
                 ground_truth_file_path, 
                 num_frames = 1000, 
                 focal_length = 718.8560, 
                 principal_axis = (607.1928, 185.2157), 
                 lk_params = dict(winSize  = (21,21), criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))):

        self.path = image_path
        self.ground_truth_path = ground_truth_file_path
        self.num_frames = num_frames
        self.focal_length = focal_length
        self.principal_axis = principal_axis
        self.lk_params = lk_params
        self.rot_matrix = np.zeros(shape = (3,3))
        self.trans_vec = np.zeros(shape = (3,3)) ##CHANGE HERE
        self.frame_id = 0
        self.num_features = 0
        self.detector = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
        self.points_to_track_pos_0 = None
        self.points_to_track_pos_1 = None
        

        frame_1_path = self.path + str(self.frame_id).zfill(6) + ".png"
        print(frame_1_path)
        self.frame1= cv2.imread(frame_1_path, 0)
        cv2.imshow('frame', self.frame1)
        cv2.waitKey(1)
    
        self.frame2 = None

        with open(self.ground_truth_path) as f:
            self.pose = f.readlines()

        

    def video_stream(self):
        if(self.frame2 is not None):
            self.frame1 = self.frame2 

        self.frame_id += 1
        frame_2_path = self.path + str(self.frame_id).zfill(6) + ".png"
        self.frame2 = cv2.imread(frame_2_path, 0)

    def visual_odometry(self):

        if (self.num_features < 2000):
            points = self.detector.detect(self.frame1)
            self.points_to_track_pos_0 = np.array([x.pt for x in points], dtype=np.float32).reshape(-1, 1, 2)
       
        self.points_to_track_pos_1, st, err = cv2.calcOpticalFlowPyrLK(self.frame1, self.frame2, self.points_to_track_pos_0, None, **self.lk_params)
       
        self.old_good_points = self.points_to_track_pos_0[st == 1]
        self.new_good_points = self.points_to_track_pos_1[st == 1]          
        E, _ = cv2.findEssentialMat(self.new_good_points,self.old_good_points,self.focal_length,self.principal_axis,cv2.RANSAC,0.999,1.0,None)
        if (self.frame_id <= 2):
            _,self.rot_matrix,self.trans_vec, _ = cv2.recoverPose(E, self.old_good_points, self.new_good_points,self.rot_matrix,self.trans_vec,self.focal_length,self.principal_axis,None)
        else:
            _, R, t, _ = cv2.recoverPose(E, self.old_good_points,self.new_good_points, self.rot_matrix.copy(), self.trans_vec.copy(), self.focal_length, self.principal_axis, None)

        ground_truth_translation = self.get_ground_truth_tranlation()
        
        if (self.frame_id>2 and ground_truth_translation > 0.1 and abs(t[2][0]) > abs(t[0][0]) and abs(t[2][0]) > abs(t[1][0])):
            self.trans_vec = self.trans_vec + ground_truth_translation*self.rot_matrix.dot(t)
            self.rot_matrix = R.dot(self.rot_matrix)

    def get_ground_truth_tranlation(self):
       
        prev_pose = self.pose[self.frame_id-1].strip().split()
        self.prev_pose = np.array([[prev_pose[3]],
                              [prev_pose[7]],
                              [prev_pose[11]]]).astype(float)
        
        current_pose = self.pose[self.frame_id].strip().split()
        self.current_pose = np.array([[current_pose[3]],
                                 [current_pose[7]],
                                 [current_pose[11]]]).astype(float)
        return np.linalg.norm(self.current_pose-self.prev_pose)

In [6]:
img_path = 'C:\\Users\\madha\\Pictures\\00\\image_0\\'

ground_truth_file_path = r"C:\Users\madha\Pictures\00\00.txt" 

vo = MonoVO(img_path,ground_truth_file_path)
output_trajectory = np.zeros((600,800,3))

for frame in range(1000):
    img = vo.frame1
    cv2.imshow('frame', img)
    k = cv2.waitKey(1)
    if k == 27:
        break
    vo.video_stream()
    vo.visual_odometry()
    
    vo_coordinates =np.matmul(-1*(np.eye(3)), vo.trans_vec) #to get the same coordinate system
    ground_truth = vo.current_pose.flatten() 

    print("Error: ",np.linalg.norm(vo_coordinates - ground_truth))
    
    output_trajectory = cv2.circle(output_trajectory, (int(vo_coordinates[0]+300),int(vo_coordinates[2]+100)),1,(0,0,255),4)
    
    cv2.imshow('trajectory', output_trajectory)
cv2.destroyAllWindows()

C:\Users\madha\Pictures\00\image_0\000000.png
Error:  1.9176613558730642
Error:  3.001338857707252
Error:  4.722934175535795
Error:  6.440367076618904
Error:  8.190019646472882
Error:  9.944454398262174
Error:  11.695240664807653
Error:  13.471797738366652
Error:  15.257857368445308
Error:  17.021778892413472
Error:  18.797460711410604
Error:  20.56990418580312
Error:  22.340622246504793
Error:  24.13161554320499
Error:  25.899130954663203
Error:  27.68862077011905
Error:  29.489817452269023
Error:  31.342244762140158
Error:  33.1936004395054
Error:  35.0626988061078
Error:  36.93573704091161
Error:  38.84419828282863
Error:  40.758099358238674
Error:  42.65203896885507
Error:  44.5789951741991
Error:  46.52844035333085
Error:  48.467034893688236
Error:  50.41965961884279
Error:  52.374238434938356
Error:  54.329545714893705
Error:  56.29731147771358
Error:  58.28925618540229
Error:  60.29638414499971
Error:  62.307338508595656
Error:  64.3316656310757
Error:  66.34569203361035
Error: 